In [1]:
import pandas as pd
import os
import fasttext
import progressbar
import json
import gzip
import shutil
fasttext.FastText.eprint = lambda x: None # avoid Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

In [4]:
path_parent = os.path.dirname(os.getcwd())
pretrained_fasttext_path = os.path.join(path_parent, 'src/models/lid.176.bin')
model = fasttext.load_model(pretrained_fasttext_path)

In [11]:
text = "This$ is a may2be an english 2text"

In [13]:
model.predict(text, threshold = 0.6)

((), array([], dtype=float64))

In [5]:
def remove_with_fasttext():
    """
    reads all files from cleaned data path and removes non-english products from the data tables
    :return:
    """
    pretrained_fasttext_path = os.path.join(path_parent, 'src/models/lid.176.bin')
    model = fasttext.load_model(pretrained_fasttext_path)

    # list all top 100 product files
    files = [file for file in os.listdir(cleaned_data_path) if file.endswith('.json.gz')]

    removed_rows_dict = {}

    count_files = 0

    for file in files:
        print(file)
        df = pd.read_json(os.path.join(cleaned_data_path, '{}'.format(file)), compression='gzip', lines=True)
        if df.shape[0] > 20: # for top100 & min3
        #if df.shape[0] > 0: # for rest only
            df['concat'] = ''

            for j in range(df.shape[1]):  # iterate over columns
                df['concat'] = df['concat'] + df.iloc[:,j].astype('str')

            # iterrate over rows and save row_ids of english products
            english_products = []
            non_english_products = []
            count = 0
            with progressbar.ProgressBar(max_value=df.shape[0]) as bar:
                for i in range(df.shape[0]):  # iterate over rows
                    row_id = int(df['row_id'][i])
                    cell = df['concat'][i]
                    cell_pred = model.predict([cell])[0]
                    print(cell_pred, model-predict([cell])[1])
                    if cell_pred == [['__label__en']]:
                        english_products.append(row_id)
                    else:
                        non_english_products.append(row_id)
                    count += 1
                    bar.update(count)

            # drop concatendated row again
            df = df.drop('concat', axis=1)

            # write removed row_ids to dictionary
            removed_rows_dict[file] = non_english_products

            # design new dataframe with english products only
            df_cleaned = df[df['row_id'].isin(english_products)]

            # write to gzip compressed json file
            if df_cleaned.shape[0] > 20: # for top100 & min3
            #if df_cleaned.shape[0] > 0: # for rest only
                df_cleaned.to_json(os.path.join(cleaned_data_path, '{}'.format(file)), compression='gzip', orient='records', lines=True)
            else:
                os.remove(os.path.join(cleaned_data_path, '{}'.format(file)))


        else:
            # if df does not contain more than 20 entries delete it from cleaned file path
            os.remove(os.path.join(cleaned_data_path, '{}'.format(file)))

        count_files += 1
        print('{} out of {} files done'.format(count_files, len(files)))

    # save dictionary with removed row_ids
    with open(os.path.join(cleaned_data_path, 'removed_rows_dict.json'), 'w', encoding='utf-8') as f:
        json.dump(removed_rows_dict, f)

    print('removed_rows_dict saved')


In [6]:
data_path = os.path.join(path_parent, 'src/data/product/LocalBusiness_top100')
cleaned_data_path = os.path.join(data_path, 'cleaned_new_threshold')
remove_with_fasttext()

FileNotFoundError: [Errno 2] No such file or directory: '/work-ceph/bizer-tp2021/data_integration_using_deep_learning/src/data/product/LocalBusiness_top100/cleaned_new_threshold'